In [1]:
import pandas as pd

market = pd.read_csv("data/processed/market_health_scores.csv")
dist = pd.read_csv("data/processed/distribution_health_scores.csv")
supply = pd.read_csv("data/processed/supply_sustainability_scores.csv")

df = market.merge(
    dist[["coingecko_id","distribution_health_score"]],
    on="coingecko_id"
).merge(
    supply[["coingecko_id","supply_sustainability_score"]],
    on="coingecko_id"
)

df.head()


,coingecko_id,pulled_at_utc,current_price_usd,market_cap_usd,volume_24h_usd,circulating_supply,max_supply,fdv_usd,token_name_x,symbol_x,...,peer_group,vol_score,drawdown_score,liquidity_score,trend_stability_score,market_health_raw,penalty,market_health_score,distribution_health_score,supply_sustainability_score
0,uniswap,2026-01-19T12:34:30.906620+00:00,4.980000,3.160631e+09,382896158,6.346104e+08,1.000000e+09,4480843112,Uniswap,UNI,...,A_DEX,0.0,100.0,100.0,98.198445,69.729767,0,69.729767,55.0,46.666667
1,chainlink,2026-01-19T12:34:45.019802+00:00,12.750000,9.024027e+09,648920230,7.081000e+08,1.000000e+09,12744001030,Chainlink,LINK,...,A_Infrastructure & Middleware,0.0,100.0,100.0,98.629791,69.794469,0,69.794469,55.0,52.833333
2,aave,2026-01-19T12:34:47.605564+00:00,163.460000,2.481823e+09,375268933,1.518357e+07,1.600000e+07,2615272322,Aave,AAVE,...,A_Lending & Borrowing,0.0,50.0,100.0,98.487477,54.773122,0,54.773122,57.5,65.833333
3,lido-dao,2026-01-19T12:35:14.316675+00:00,0.546995,4.629709e+08,56720533,8.465668e+08,1.000000e+09,546880520,Lido Dao,LDO,...,A_Liquid staking,0.0,100.0,100.0,96.686400,69.502960,0,69.502960,55.0,56.666667
4,curve-dao-token,2026-01-19T12:35:17.069355+00:00,0.390634,5.737410e+08,74930533,1.468781e+09,3.030303e+09,915416077,Curve DAO,CRV,...,B_DEX,0.0,100.0,100.0,98.988879,69.848332,0,69.848332,77.5,42.833333


In [2]:
df["tokenomics_health_score"] = (
    0.40 * df["market_health_score"] +
    0.30 * df["distribution_health_score"] +
    0.30 * df["supply_sustainability_score"]
).clip(0, 100)


In [3]:
def health_grade(score):
    if score >= 80:
        return "A (Healthy)"
    elif score >= 65:
        return "B (Solid)"
    elif score >= 50:
        return "C (Watch)"
    elif score >= 35:
        return "D (Risky)"
    else:
        return "F (High Risk)"

df["health_grade"] = df["tokenomics_health_score"].apply(health_grade)


In [4]:
df[[
    "coingecko_id",
    "tokenomics_health_score",
    "health_grade",
    "market_health_score",
    "distribution_health_score",
    "supply_sustainability_score"
]].sort_values("tokenomics_health_score", ascending=False)


,coingecko_id,tokenomics_health_score,health_grade,market_health_score,distribution_health_score,supply_sustainability_score
5,havven,69.586822,B (Solid),69.592054,55.0,84.166667
11,basic-attention-token,68.127656,B (Solid),69.944139,55.0,78.833333
6,compound-governance-token,66.839895,B (Solid),69.599738,55.0,75.000000
10,yearn-finance,65.734416,B (Solid),69.711041,55.0,71.166667
4,curve-dao-token,64.039333,C (Watch),69.848332,77.5,42.833333
16,1inch,62.990326,C (Watch),69.725815,55.0,62.000000
3,lido-dao,61.301184,C (Watch),69.502960,55.0,56.666667
1,chainlink,60.267787,C (Watch),69.794469,55.0,52.833333
12,pepe,59.686894,C (Watch),54.717235,35.0,91.000000
2,aave,58.909249,C (Watch),54.773122,57.5,65.833333


In [5]:
import os
os.makedirs("data/processed", exist_ok=True)

df.to_csv(
    "data/processed/tokenomics_health_engine_output.csv",
    index=False
)

"data/processed/tokenomics_health_engine_output.csv"


'data/processed/tokenomics_health_engine_output.csv'